In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
import re

In [2]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)

In [3]:
str_columns = ['product_title', 'search_term', 'orig_search_term',
               'orig_product_title', 'descr', 'orig_descr', 'brand']
str_culomns_dict = dict(zip(str_columns, ['str']*len(str_columns)))

train = pd.read_csv('zip/train4.csv.gz', index_col=['id'], compression='gzip', dtype=str_culomns_dict)
train.head()

,product_uid,product_title,search_term,relevance,orig_search_term,orig_product_title,descr,orig_descr,match_product_title,match_descr,len_search_term_words,len_search_term_letters,complete_match_title,complete_match_descr,brand,match_brand,words_std_title,words_std_descr,last_word_title,last_word_descr,prod_title_ind,descr_ind,match_pos_title,match_pos_descr,match_orig_title,match_orig_descr,len_title,len_descr,fuzzy_title,fuzzy_descr,fuzzy_orig_title,fuzzy_orig_descr,part_fuzzy_title,part_fuzzy_descr,part_fuzzy_orig_title,part_fuzzy_orig_descr,pred_diff,first_word_title,first_word_descr,match_numbers_title,match_numbers_descr
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,100001,simpson strong tie 12 gaug angl,angl bracket,3.00,angle bracket,Simpson Strong-Tie 12-Gauge Angle,"not angl make joint stronger, also provid consistent, straight corners. simpson strong-ti offer wide varieti angl various size thick handl light-duti job project structur connect needed. some bent (skewed) match project. for outdoor project moistur present, use zmax zinc-coat connectors, provid extra resist corros (look ""z"" end model number).versatil connector various 90 connect home repair projectsstrong angl nail screw fasten alonehelp ensur joint consist straight strongdimensions: 3 in. x 3 in. x 1-1/2 in.mad 12-gaug steelgalvan extra corros resistanceinstal 10d common nail #9 x 1-1/2 in. strong-driv sd screw","Not only do angles make joints stronger, they also provide more consistent, straight corners. Simpson Strong-Tie offers a wide variety of angles in various sizes and thicknesses to handle light-duty jobs or projects where a structural connection is needed. Some can be bent (skewed) to match the project. For outdoor projects or those where moisture is present, use our ZMAX zinc-coated connectors, which provide extra resistance against corrosion (look for a ""Z"" at the end of the model number).Versatile connector for various 90 connections and home repair projectsStronger than angled nailing or screw fastening aloneHelp ensure joints are consistently straight and strongDimensions: 3 in. x 3 in. x 1-1/2 in.Made from 12-Gauge steelGalvanized for extra corrosion resistanceInstall with 10d common nails or #9 x 1-1/2 in. Strong-Drive SD screws",0.500000,0.500000,2,12,0,0,simpson strong-ti,0.0,0.00,0.00,0,0,0.833333,0.011236,0.500000,0.000000,0.0,0.0,6,89,50,50,56,1,100,100,100,62,0.185792,1,1,0,0
3,100001,simpson strong tie 12 gaug angl,l bracket,2.50,l bracket,Simpson Strong-Tie 12-Gauge Angle,"not angl make joint stronger, also provid consistent, straight corners. simpson strong-ti offer wide varieti angl various size thick handl light-duti job project structur connect needed. some bent (skewed) match project. for outdoor project moistur present, use zmax zinc-coat connectors, provid extra resist corros (look ""z"" end model number).versatil connector various 90 connect home repair projectsstrong angl nail screw fasten alonehelp ensur joint consist straight strongdimensions: 3 in. x 3 in. x 1-1/2 in.mad 12-gaug steelgalvan extra corros resistanceinstal 10d common nail #9 x 1-1/2 in. strong-driv sd screw","Not only do angles make joints stronger, they also provide more consistent, straight corners. Simpson Strong-Tie offers a wide variety of angles in various sizes and thicknesses to handle light-duty jobs or projects where a structural connection is needed. Some can be bent (skewed) to match the project. For outdoor projects or those where moisture is present, use our ZMAX zinc-coated connectors, which provide extra resistance against corrosion (look for a ""Z"" at the end of the model number).Versatile connector for various 90 connections and home repair projectsStronger than angled nailing or screw fastening aloneHelp ensure joints are consistently straight and strongDimensions: 3 in. x 3 in. x 1-1/2 in.Made from 12-Gauge steelGalvanized for extra corrosion resistanceInstall with 10d common nails or #9 x 1-1/2 in. Strong-Drive SD screws",0.00000

In [4]:
from fuzzywuzzy import fuzz

@np.vectorize
def fuzzy_match(search_term, field):
    if search_term == 'null':
        return 0
    return fuzz.token_set_ratio(search_term, field)



@np.vectorize
def word_match_count(search_term, text):
    count = 0
    search_term = search_term.split()
    text = text.split()
    if len(search_term) == 0:
        return 0
    for word in set(search_term):
        count += 1 if text.count(word) > 0 else 0
    return count / len(search_term)

@np.vectorize
def fuzzy_match_partial(search_term, field):
    if search_term == 'null':
        return 0
    return fuzz.partial_token_set_ratio(search_term, field)

In [5]:
train['match_orig_title'] = word_match_count(train['orig_search_term'].str.lower(),
                                             train['orig_product_title'].str.lower())
train['match_orig_descr'] = word_match_count(train['orig_search_term'].str.lower(),
                                             train['orig_descr'].str.lower())

train['fuzzy_brand'] = fuzzy_match_partial(train['search_term'], train['brand'])
train.head()

,product_uid,product_title,search_term,relevance,orig_search_term,orig_product_title,descr,orig_descr,match_product_title,match_descr,len_search_term_words,len_search_term_letters,complete_match_title,complete_match_descr,brand,match_brand,words_std_title,words_std_descr,last_word_title,last_word_descr,prod_title_ind,descr_ind,match_pos_title,match_pos_descr,match_orig_title,match_orig_descr,len_title,len_descr,fuzzy_title,fuzzy_descr,fuzzy_orig_title,fuzzy_orig_descr,part_fuzzy_title,part_fuzzy_descr,part_fuzzy_orig_title,part_fuzzy_orig_descr,pred_diff,first_word_title,first_word_descr,match_numbers_title,match_numbers_descr,fuzzy_brand
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,100001,simpson strong tie 12 gaug angl,angl bracket,3.00,angle bracket,Simpson Strong-Tie 12-Gauge Angle,"not angl make joint stronger, also provid consistent, straight corners. simpson strong-ti offer wide varieti angl various size thick handl light-duti job project structur connect needed. some bent (skewed) match project. for outdoor project moistur present, use zmax zinc-coat connectors, provid extra resist corros (look ""z"" end model number).versatil connector various 90 connect home repair projectsstrong angl nail screw fasten alonehelp ensur joint consist straight strongdimensions: 3 in. x 3 in. x 1-1/2 in.mad 12-gaug steelgalvan extra corros resistanceinstal 10d common nail #9 x 1-1/2 in. strong-driv sd screw","Not only do angles make joints stronger, they also provide more consistent, straight corners. Simpson Strong-Tie offers a wide variety of angles in various sizes and thicknesses to handle light-duty jobs or projects where a structural connection is needed. Some can be bent (skewed) to match the project. For outdoor projects or those where moisture is present, use our ZMAX zinc-coated connectors, which provide extra resistance against corrosion (look for a ""Z"" at the end of the model number).Versatile connector for various 90 connections and home repair projectsStronger than angled nailing or screw fastening aloneHelp ensure joints are consistently straight and strongDimensions: 3 in. x 3 in. x 1-1/2 in.Made from 12-Gauge steelGalvanized for extra corrosion resistanceInstall with 10d common nails or #9 x 1-1/2 in. Strong-Drive SD screws",0.500000,0.500000,2,12,0,0,simpson strong-ti,0.0,0.00,0.00,0,0,0.833333,0.011236,0.500000,0.000000,0.500000,0.000000,6,89,50,50,56,1,100,100,100,62,0.185792,1,1,0,0,33
3,100001,simpson strong tie 12 gaug angl,l bracket,2.50,l bracket,Simpson Strong-Tie 12-Gauge Angle,"not angl make joint stronger, also provid consistent, straight corners. simpson strong-ti offer wide varieti angl various size thick handl light-duti job project structur connect needed. some bent (skewed) match project. for outdoor project moistur present, use zmax zinc-coat connectors, provid extra resist corros (look ""z"" end model number).versatil connector various 90 connect home repair projectsstrong angl nail screw fasten alonehelp ensur joint consist straight strongdimensions: 3 in. x 3 in. x 1-1/2 in.mad 12-gaug steelgalvan extra corros resistanceinstal 10d common nail #9 x 1-1/2 in. strong-driv sd screw","Not only do angles make joints stronger, they also provide more consistent, straight corners. Simpson Strong-Tie offers a wide variety of angles in various sizes and thicknesses to handle light-duty jobs or projects where a structural connection is needed. Some can be bent (skewed) to match the project. For outdoor projects or those where moisture is present, use our ZMAX zinc-coated connectors, which provide extra resistance against corrosion (look for a ""Z"" at the end of the model number).Versatile connector for various 90 connections and home repair projectsStronger than angled nailing or screw fastening aloneHelp ensure joints are consistently straight and strongDimensions: 3 in. x 3 in. x 1-1/2 in.Made from 12-Gauge steelGalvanized for extra corrosion resistanceInstall with 10d common nails or #9 x 1-1/2 in. Stron

In [6]:
test = pd.read_csv('zip/test4.csv.gz', index_col=['id'], compression='gzip', dtype=str_culomns_dict)
test['match_orig_title'] = word_match_count(test['orig_search_term'].str.lower(),
                                            test['orig_product_title'].str.lower())
test['match_orig_descr'] = word_match_count(test['orig_search_term'].str.lower(),
                                            test['orig_descr'].str.lower())

test['fuzzy_brand'] = fuzzy_match_partial(test['search_term'], test['brand'])
test.head()

,product_uid,product_title,search_term,orig_search_term,orig_product_title,descr,orig_descr,match_product_title,match_descr,len_search_term_words,len_search_term_letters,complete_match_title,complete_match_descr,brand,match_brand,words_std_title,words_std_descr,last_word_title,last_word_descr,match_pos_title,match_pos_descr,match_orig_title,match_orig_descr,len_title,len_descr,prod_title_ind,descr_ind,fuzzy_title,fuzzy_descr,fuzzy_orig_title,fuzzy_orig_descr,part_fuzzy_title,part_fuzzy_descr,part_fuzzy_orig_title,part_fuzzy_orig_descr,first_word_title,first_word_descr,match_numbers_title,match_numbers_descr,fuzzy_brand
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,100001,simpson strong tie 12 gaug angl,90deg. bracket,90 degree bracket,Simpson Strong-Tie 12-Gauge Angle,"not angl make joint stronger, also provid consistent, straight corners. simpson strong-ti offer wide varieti angl various size thick handl light-duti job project structur connect needed. some bent (skewed) match project. for outdoor project moistur present, use zmax zinc-coat connectors, provid extra resist corros (look ""z"" end model number).versatil connector various 90 connect home repair projectsstrong angl nail screw fasten alonehelp ensur joint consist straight strongdimensions: 3 in. x 3 in. x 1-1/2 in.mad 12-gaug steelgalvan extra corros resistanceinstal 10d common nail #9 x 1-1/2 in. strong-driv sd screw","Not only do angles make joints stronger, they also provide more consistent, straight corners. Simpson Strong-Tie offers a wide variety of angles in various sizes and thicknesses to handle light-duty jobs or projects where a structural connection is needed. Some can be bent (skewed) to match the project. For outdoor projects or those where moisture is present, use our ZMAX zinc-coated connectors, which provide extra resistance against corrosion (look for a ""Z"" at the end of the model number).Versatile connector for various 90 connections and home repair projectsStronger than angled nailing or screw fastening aloneHelp ensure joints are consistently straight and strongDimensions: 3 in. x 3 in. x 1-1/2 in.Made from 12-Gauge steelGalvanized for extra corrosion resistanceInstall with 10d common nails or #9 x 1-1/2 in. Strong-Drive SD screws",0.000000,0.000000,2,14,0,0,simpson strong-ti,0.0,0,0,0,0,0.000000,0.000000,0.000000,0.333333,6,89,1.0,1.00000,18,2,12,21,23,38,35,100,0,0,0.0,1.0,23
4,100001,simpson strong tie 12 gaug angl,metal l bracket,metal l brackets,Simpson Strong-Tie 12-Gauge Angle,"not angl make joint stronger, also provid consistent, straight corners. simpson strong-ti offer wide varieti angl various size thick handl light-duti job project structur connect needed. some bent (skewed) match project. for outdoor project moistur present, use zmax zinc-coat connectors, provid extra resist corros (look ""z"" end model number).versatil connector various 90 connect home repair projectsstrong angl nail screw fasten alonehelp ensur joint consist straight strongdimensions: 3 in. x 3 in. x 1-1/2 in.mad 12-gaug steelgalvan extra corros resistanceinstal 10d common nail #9 x 1-1/2 in. strong-driv sd screw","Not only do angles make joints stronger, they also provide more consistent, straight corners. Simpson Strong-Tie offers a wide variety of angles in various sizes and thicknesses to handle light-duty jobs or projects where a structural connection is needed. Some can be bent (skewed) to match the project. For outdoor projects or those where moisture is present, use our ZMAX zinc-coated connectors, which provide extra resistance against corrosion (look for a ""Z"" at the end of the model number).Versatile connector for various 90 connections and home repair projectsStronger than angled nailing or screw fastening aloneHelp ensure joints are consistently straight and strongDimensions: 3 in. x 3 in. x 1-1/2 in.Made from 12-Gauge steelGalvanized for extra corrosion resistanceInstall with 10d common nails or #9 x 1-1/2 in. Strong-Drive SD screws",0.000000,0.00

In [7]:
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion

class cust_regression_vals(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self
    def transform(self, train):
        return train[tree_cols].values


class cust_txt_col(BaseEstimator, TransformerMixin):
    def __init__(self, col):
        self.col = col
    def fit(self, x, y=None):
        return self
    def transform(self, dataset):
        return dataset[self.col].apply(str)

def fitter():
    clf = Pipeline([
    ('union', FeatureUnion(
                transformer_list = [
                    ('cst',  cust_regression_vals()),  
                    ('txt1', Pipeline([('s1', cust_txt_col(col='search_term')),
                                       ('tfidf1', TfidfVectorizer(stop_words='english')),
                                       ('tsvd1', TruncatedSVD(n_components=80))])),
                    ('txt2', Pipeline([('s2', cust_txt_col(col='product_title')),
                                       ('tfidf2', TfidfVectorizer(stop_words='english')),
                                       ('tsvd2', TruncatedSVD(n_components=80))])),
                    ('txt3', Pipeline([('s3', cust_txt_col(col='descr')),
                                       ('tfidf3', TfidfVectorizer(stop_words='english')),
                                       ('tsvd3', TruncatedSVD(n_components=60))])),
                    ('txt4', Pipeline([('s4', cust_txt_col(col='brand')),
                                       ('tfidf4', TfidfVectorizer(stop_words='english')),
                                       ('tsvd4', TruncatedSVD(n_components=30))]))
                    ]
            ))])
    clf.fit(train[use_cols])
    return clf

def write_submission(model, test_data):
    y_pred = model.predict(test_data)
    y_pred[y_pred>3.] = 3.
    y_pred[y_pred<1.] = 1.
    results = pd.DataFrame({'id':test.index.values, 'relevance':y_pred})
    results.to_csv('lgbm5.csv', header=True, index=False)

In [8]:
tree_cols = ['match_product_title', 'match_descr', 'len_search_term_words', 'len_search_term_letters',
             'match_pos_title', 'match_pos_descr', 'prod_title_ind', 'descr_ind',
             'words_std_title', 'words_std_descr', 'len_title', 'len_descr', 'last_word_title',
             'last_word_descr', 'match_orig_title', 'match_orig_descr',
             'fuzzy_title', 'fuzzy_descr', 'fuzzy_orig_title', 'fuzzy_orig_descr',
             'part_fuzzy_title', 'part_fuzzy_descr', 'part_fuzzy_orig_title', 'part_fuzzy_orig_descr',
             'first_word_title', 'first_word_descr',
             'match_numbers_title',
             'fuzzy_brand']

use_cols = tree_cols + ['search_term', 'product_title', 'descr', 'brand']

clf = fitter()
X_train = clf.transform(train[use_cols])
X_test = clf.transform(test[use_cols])
y = train['relevance'].values

gbm = lgb.LGBMRegressor(learning_rate=0.03, colsample_bytree=0.5, num_leaves=80, n_estimators=1000)
gbm.fit(X_train, y)

tc_len = len(tree_cols)
print(dict(zip(tree_cols, gbm.feature_importances_[:tc_len]) ))

write_submission(gbm, X_test)

{'match_product_title': 243, 'match_descr': 168, 'len_search_term_words': 240, 'len_search_term_letters': 620, 'match_pos_title': 168, 'match_pos_descr': 128, 'prod_title_ind': 567, 'descr_ind': 485, 'words_std_title': 244, 'words_std_descr': 261, 'len_title': 385, 'len_descr': 400, 'last_word_title': 102, 'last_word_descr': 63, 'match_orig_title': 223, 'match_orig_descr': 147, 'fuzzy_title': 609, 'fuzzy_descr': 367, 'fuzzy_orig_title': 494, 'fuzzy_orig_descr': 361, 'part_fuzzy_title': 160, 'part_fuzzy_descr': 164, 'part_fuzzy_orig_title': 215, 'part_fuzzy_orig_descr': 69, 'first_word_title': 94, 'first_word_descr': 59, 'match_numbers_title': 142, 'fuzzy_brand': 303}


0.46769

In [9]:
train['pred_diff'] = train['relevance'] - gbm.predict(X_train)
train.sort_values(['pred_diff'], ascending=False)[:50]

,product_uid,product_title,search_term,relevance,orig_search_term,orig_product_title,descr,orig_descr,match_product_title,match_descr,len_search_term_words,len_search_term_letters,complete_match_title,complete_match_descr,brand,match_brand,words_std_title,words_std_descr,last_word_title,last_word_descr,prod_title_ind,descr_ind,match_pos_title,match_pos_descr,match_orig_title,match_orig_descr,len_title,len_descr,fuzzy_title,fuzzy_descr,fuzzy_orig_title,fuzzy_orig_descr,part_fuzzy_title,part_fuzzy_descr,part_fuzzy_orig_title,part_fuzzy_orig_descr,pred_diff,first_word_title,first_word_descr,match_numbers_title,match_numbers_descr,fuzzy_brand
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
203914,192088,behr premium plus 600e 2 harbor mist zero voc interior paint,locit 2 plus 1,3.00,locite 2 plus 1,BEHR Premium Plus #600E-2 Harbor Mist Zero VOC Interior Paint,"behr premium plus zero voc, self-prim interior flat 100% acryl provid mildew resist finish. this easy-clean flat provid excel touch-up, ultim durabl except hide. the finish display non-reflective, matt appearance.california residents: see&nbsp;proposit 65 informationid famili rooms, live rooms, dine rooms, bedroom ceilings100% acryl finish easi cleaningtough durabl matt finishexcept hide, 1 cover 400 sq. ft. depend color select surfac porosityeasi soap water clean-upexcept latex formula provid mildew resist finishself prime finish seal surfaceactu paint color may vari on-screen printer representationsonlin price includ paint care fee follow states: ca, co, ct, me, mn, or, ri, vt","BEHR Premium Plus Zero VOC, Self-Priming Interior Flat is 100% acrylic and provides a mildew resistant finish. This easy-clean flat provides excellent touch-up, ultimate durability and exceptional hide. The finish displays a non-reflective, matte appearance.California residents: see&nbsp;Proposition 65 informationIdeal for family rooms, living rooms, dining rooms, bedrooms and ceilings100% acrylic finish for easy cleaningTough and durable matte finishExceptional hide, 1 can covers up to 400 sq. ft. depending on color selected and surface porosityEasy soap and water clean-upExceptional latex formula provides a mildew resistant finishSelf priming finish that seals the surfaceActual paint colors may vary from on-screen and printer representationsOnline Price includes Paint Care fee in the following states: CA, CO, CT, ME, MN, OR, RI, VT",0.500000,0.500000,4,14,0,0,behr premium plus,0.333333,1.00,576.00,0,1,0.181818,0.021053,0.500000,0.500000,0.250000,0.500000,11,95,60,60,57,57,100,100,100,100,1.165816,0,0,0,0,100
207064,194599,home legend distress lennox hickori 3 8in. thick xbi 2 1 8in. wide xbi 78in. length hardwood carpet reduc mold,ennox,3.00,ennox,Home Legend Distressed Lennox Hickory 3/8 in. Thick x 2-1/8 in. Wide x 78 in. Length Hardwood Carpet Reducer Molding,"78 in. length, hardwood veneer carpet reduc overlap molding. it use provid smooth safe transit floor cover differ height constructions. it also use border around fire place hearth slide door threshold. the prefinish mold nail glu coordin style floor.california residents: see&nbsp;proposit 65 informationwood species: hickory3/8 in. thick x 2-1/8 in. width x 78 in. length moldingitem coordin 3/8 in. thick hardwood flooringconstruction: high densiti fiberboard (hdf) moldingprefinish lennox hickory, dark colorjanka wood hard rating/resist denting: 1820appropri grade installation: all gradesinstallation: glueresidenti useal onlin order item ship via parcel ground may arriv multipl box","78 in. length, hardwood veneer carpet reducer is an overlap molding. It is used to provide a smooth and safe transition between floor coverings of different heights and constructions. It can also be used to as borders around fire place hearth and at sliding door threshold. The prefinished molding can be nailed or glued with its coordinating style of floor.California residents: see&nbsp;Proposition 65 informationWood Species: Hickory3/8 in. Thickness x 2-1/8 in. W

In [10]:
train.sort_values(['pred_diff'], ascending=True)[:50]

,product_uid,product_title,search_term,relevance,orig_search_term,orig_product_title,descr,orig_descr,match_product_title,match_descr,len_search_term_words,len_search_term_letters,complete_match_title,complete_match_descr,brand,match_brand,words_std_title,words_std_descr,last_word_title,last_word_descr,prod_title_ind,descr_ind,match_pos_title,match_pos_descr,match_orig_title,match_orig_descr,len_title,len_descr,fuzzy_title,fuzzy_descr,fuzzy_orig_title,fuzzy_orig_descr,part_fuzzy_title,part_fuzzy_descr,part_fuzzy_orig_title,part_fuzzy_orig_descr,pred_diff,first_word_title,first_word_descr,match_numbers_title,match_numbers_descr,fuzzy_brand
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
69679,119478,romano 4ft. boxwood spiral topiari tree,topiari tree,1.00,topiary tree,Romano 4 ft. Boxwood Spiral Topiary Tree,"enhanc home romano boxwood spiral topiari tree. this wonder full bodi boxwood spiral topiari tree keep color rest lose color leaves. you alway look forward rich green tone romano boxwood spiral topiari tree home. if look high qualiti artifici tree make bold statement home, it.light mobil designov 700 leavesindoor/outdooriron frameuv resist rubber leaves7 in. potenh room hous","Enhance your home with a Romano Boxwood Spiral Topiary Tree. This wonderful full bodied Boxwood Spiral Topiary Tree keeps its colors when all the rest lose their color and leaves. You can always look forward to the rich green tones of the Romano Boxwood Spiral Topiary Tree in your home. If you are looking for a High quality artificial tree to make a bold statement in your home, this is it.Light and mobile designOver 700 leavesIndoor/outdoorIron frameUV resistant rubber leaves7 in. potEnhances any room in your house",1.000000,1.000000,2,12,1,1,NaN,0.0,0.250000,20.250000,1,1,0.666667,0.087719,1.000000,1.000000,1.000000,1.000000,6,57,100,100,100,100,100,100,100,100,-1.605070,1,1,0,0,33
156918,160006,werner 14ft. fiberglass round rung straight ladder 375lb. load capac type iaa duti rate,14ft. ladder,1.00,14 ft ladder,Werner 14 ft. Fiberglass Round Rung Straight Ladder with 375 lb. Load Capacity Type IAA Duty Rating,"the 7114-1 one-sect round rung 14 ft. straight ladder made non-conduct fiberglass rails. easi transport maneuver, ladder bear duti rate 375 lbs. make durabl long lasting. the slip-resistant, traction-tr rung round, durabl rail shield shoe bracket help protect rail damage. the plate rung assembl rivet four point side rails.alflo twist-proof performancemar resist end capsslip resist round rungssho pad spur platecannot ship home, avail store deliveri","The 7114-1 one-section round rung 14 ft. Straight Ladder is made with non-conductive fiberglass rails. Easy to transport and maneuver, these ladders bear a duty rating of 375 lbs. making them durable and long lasting. The slip-resistant, TRACTION-TRED rungs are round, and the durable rail shields and shoe brackets help to protect the rails from damage. The plate and rung assemblies are riveted at four points to the side rails.ALFLO TWIST-PROOF performanceMar resistant end capsSlip resistant round rungsShoe with pad and spur plateCannot ship to home, available for store delivery only",1.000000,0.500000,2,12,0,0,NaN,0.0,6.250000,0.000000,1,1,0.071429,0.123077,1.000000,1.000000,0.666667,0.666667,14,65,100,71,100,100,100,100,100,100,-1.454091,1,0,1,1,33
120624,140844,rachael ray 10 qt. cover stockpot,rachael ray,1.00,rachael ray,Rachael Ray 10 qt. Covered Stockpot,"whether boil pasta, make batch chili cook grains, large, 10 quart stockpot right size. when stovetop, two short side handl help take less space cabinet. the hard-anod construct prefer even heat distribut throughout pan sides. the nonstick cook surfac mean need worri food stick scrub dirti pan - cleanup breeze. in fact, rachael stockpot dishwash safe ad convenience.hard-anod cookwar except durable, plus heat quick evenly, reduc ""hot spots"" burn foodsa clear coat exterior make cookwar dishwash safe conveniencelong-last nonstick interior let f

In [11]:
train.loc[train['fuzzy_brand']<70, 'fuzzy_brand'] = 0
test.loc[test['fuzzy_brand']<70, 'fuzzy_brand'] = 0

clf = fitter()
X_train = clf.transform(train[use_cols])
X_test = clf.transform(test[use_cols])

gbm = lgb.LGBMRegressor(learning_rate=0.03, colsample_bytree=0.5, num_leaves=80, n_estimators=1000)
gbm.fit(X_train, y)

tc_len = len(tree_cols)
print(dict(zip(tree_cols, gbm.feature_importances_[:tc_len]) ))

write_submission(gbm, X_test)

{'match_product_title': 241, 'match_descr': 156, 'len_search_term_words': 246, 'len_search_term_letters': 647, 'match_pos_title': 153, 'match_pos_descr': 157, 'prod_title_ind': 567, 'descr_ind': 502, 'words_std_title': 238, 'words_std_descr': 226, 'len_title': 398, 'len_descr': 404, 'last_word_title': 90, 'last_word_descr': 70, 'match_orig_title': 213, 'match_orig_descr': 142, 'fuzzy_title': 576, 'fuzzy_descr': 425, 'fuzzy_orig_title': 478, 'fuzzy_orig_descr': 342, 'part_fuzzy_title': 175, 'part_fuzzy_descr': 149, 'part_fuzzy_orig_title': 199, 'part_fuzzy_orig_descr': 73, 'first_word_title': 92, 'first_word_descr': 61, 'match_numbers_title': 137, 'fuzzy_brand': 101}


In [14]:
tree_cols = ['match_product_title', 'match_descr', 'len_search_term_words', 'len_search_term_letters',
             'match_pos_title', 'match_pos_descr', 'prod_title_ind', 'descr_ind',
             'words_std_title', 'words_std_descr', 'len_title', 'len_descr', 'last_word_title',
             'last_word_descr', 'match_orig_title', 'match_orig_descr',
             'fuzzy_title', 'fuzzy_descr', 'fuzzy_orig_title', 'fuzzy_orig_descr',
             'part_fuzzy_title', 'part_fuzzy_descr', 'part_fuzzy_orig_title', 'part_fuzzy_orig_descr',
             'first_word_title', 'first_word_descr',
             'match_numbers_title',
             'fuzzy_brand']

use_cols = tree_cols + ['search_term', 'product_title', 'descr', 'brand']

clf = fitter()
X_train = clf.transform(train[use_cols])
X_test = clf.transform(test[use_cols])

params = {
    'num_leaves': [80, 120],
    'feature_fraction': [0.7, 0.9],
    'bagging_fraction': [0.7, 0.9],
    'bagging_freq': [50, 100]
}

gbm = lgb.LGBMRegressor(learning_rate=0.03, n_estimators=1000)

model = GridSearchCV(gbm, params,
                     cv=4,
                     n_jobs=1,
                     return_train_score=False,
                     verbose=2)
model.fit(X_train, y)
grid_df = pd.DataFrame(model.cv_results_)
grid_df.drop(columns=grid_df.columns[:3], inplace=True)
grid_df.sort_values(['rank_test_score'], inplace=True)
grid_df.head()

Fitting 4 folds for each of 16 candidates, totalling 64 fits
[CV] bagging_fraction=0.7, bagging_freq=50, feature_fraction=0.7, num_leaves=80 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  bagging_fraction=0.7, bagging_freq=50, feature_fraction=0.7, num_leaves=80, total=  27.6s
[CV] bagging_fraction=0.7, bagging_freq=50, feature_fraction=0.7, num_leaves=80 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   27.6s remaining:    0.0s


[CV]  bagging_fraction=0.7, bagging_freq=50, feature_fraction=0.7, num_leaves=80, total=  27.1s
[CV] bagging_fraction=0.7, bagging_freq=50, feature_fraction=0.7, num_leaves=80 
[CV]  bagging_fraction=0.7, bagging_freq=50, feature_fraction=0.7, num_leaves=80, total=  27.3s
[CV] bagging_fraction=0.7, bagging_freq=50, feature_fraction=0.7, num_leaves=80 
[CV]  bagging_fraction=0.7, bagging_freq=50, feature_fraction=0.7, num_leaves=80, total=  26.7s
[CV] bagging_fraction=0.7, bagging_freq=50, feature_fraction=0.7, num_leaves=120 
[CV]  bagging_fraction=0.7, bagging_freq=50, feature_fraction=0.7, num_leaves=120, total=  38.0s
[CV] bagging_fraction=0.7, bagging_freq=50, feature_fraction=0.7, num_leaves=120 
[CV]  bagging_fraction=0.7, bagging_freq=50, feature_fraction=0.7, num_leaves=120, total=  37.9s
[CV] bagging_fraction=0.7, bagging_freq=50, feature_fraction=0.7, num_leaves=120 
[CV]  bagging_fraction=0.7, bagging_freq=50, feature_fraction=0.7, num_leaves=120, total=  37.8s
[CV] bagging_

[CV]  bagging_fraction=0.9, bagging_freq=50, feature_fraction=0.9, num_leaves=120, total=  50.6s
[CV] bagging_fraction=0.9, bagging_freq=100, feature_fraction=0.7, num_leaves=80 
[CV]  bagging_fraction=0.9, bagging_freq=100, feature_fraction=0.7, num_leaves=80, total=  29.4s
[CV] bagging_fraction=0.9, bagging_freq=100, feature_fraction=0.7, num_leaves=80 
[CV]  bagging_fraction=0.9, bagging_freq=100, feature_fraction=0.7, num_leaves=80, total=  29.4s
[CV] bagging_fraction=0.9, bagging_freq=100, feature_fraction=0.7, num_leaves=80 
[CV]  bagging_fraction=0.9, bagging_freq=100, feature_fraction=0.7, num_leaves=80, total=  29.3s
[CV] bagging_fraction=0.9, bagging_freq=100, feature_fraction=0.7, num_leaves=80 
[CV]  bagging_fraction=0.9, bagging_freq=100, feature_fraction=0.7, num_leaves=80, total=  28.9s
[CV] bagging_fraction=0.9, bagging_freq=100, feature_fraction=0.7, num_leaves=120 
[CV]  bagging_fraction=0.9, bagging_freq=100, feature_fraction=0.7, num_leaves=120, total=  40.3s
[CV] b

[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed: 40.7min finished


,std_score_time,param_bagging_fraction,param_bagging_freq,param_feature_fraction,param_num_leaves,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,mean_test_score,std_test_score,rank_test_score
12,0.000965,0.9,100,0.7,80,"{'bagging_fraction': 0.9, 'bagging_freq': 100, 'feature_fraction': 0.7, 'num_leaves': 80}",0.268025,0.283458,0.300754,0.148838,0.250270,0.059694,1
13,0.001469,0.9,100,0.7,120,"{'bagging_fraction': 0.9, 'bagging_freq': 100, 'feature_fraction': 0.7, 'num_leaves': 120}",0.268690,0.284557,0.295076,0.151265,0.249899,0.057714,2
8,0.001825,0.9,50,0.7,80,"{'bagging_fraction': 0.9, 'bagging_freq': 50, 'feature_fraction': 0.7, 'num_leaves': 80}",0.268527,0.282144,0.298463,0.150319,0.249864,0.058440,3
14,0.001114,0.9,100,0.9,80,"{'bagging_fraction': 0.9, 'bagging_freq': 100, 'feature_fraction': 0.9, 'num_leaves': 80}",0.266335,0.281780,0.297363,0.152276,0.249440,0.057159,4
11,0.000525,0.9,50,0.9,120,"{'bagging_fraction': 0.9, 'bagging_freq': 50, 'feature_fraction': 0.9, 'num_leaves': 120}",0.266394,0.281852,0.297526,0.151880,0.249414,0.057376,5


In [15]:
grid_df

,std_score_time,param_bagging_fraction,param_bagging_freq,param_feature_fraction,param_num_leaves,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,mean_test_score,std_test_score,rank_test_score
12,0.000965,0.9,100,0.7,80,"{'bagging_fraction': 0.9, 'bagging_freq': 100, 'feature_fraction': 0.7, 'num_leaves': 80}",0.268025,0.283458,0.300754,0.148838,0.250270,0.059694,1
13,0.001469,0.9,100,0.7,120,"{'bagging_fraction': 0.9, 'bagging_freq': 100, 'feature_fraction': 0.7, 'num_leaves': 120}",0.268690,0.284557,0.295076,0.151265,0.249899,0.057714,2
8,0.001825,0.9,50,0.7,80,"{'bagging_fraction': 0.9, 'bagging_freq': 50, 'feature_fraction': 0.7, 'num_leaves': 80}",0.268527,0.282144,0.298463,0.150319,0.249864,0.058440,3
14,0.001114,0.9,100,0.9,80,"{'bagging_fraction': 0.9, 'bagging_freq': 100, 'feature_fraction': 0.9, 'num_leaves': 80}",0.266335,0.281780,0.297363,0.152276,0.249440,0.057159,4
11,0.000525,0.9,50,0.9,120,"{'bagging_fraction': 0.9, 'bagging_freq': 50, 'feature_fraction': 0.9, 'num_leaves': 120}",0.266394,0.281852,0.297526,0.151880,0.249414,0.057376,5
15,0.004512,0.9,100,0.9,120,"{'bagging_fraction': 0.9, 'bagging_freq': 100, 'feature_fraction': 0.9, 'num_leaves': 120}",0.268330,0.283591,0.297867,0.147147,0.249235,0.059857,6
9,0.003330,0.9,50,0.7,120,"{'bagging_fraction': 0.9, 'bagging_freq': 50, 'feature_fraction': 0.7, 'num_leaves': 120}",0.265635,0.284079,0.296724,0.149338,0.248946,0.058560,7
10,0.001943,0.9,50,0.9,80,"{'bagging_fraction': 0.9, 'bagging_freq': 50, 'feature_fraction': 0.9, 'num_leaves': 80}",0.265445,0.280976,0.297255,0.149778,0.248365,0.058018,8
0,0.002175,0.7,50,0.7,80,"{'bagging_fraction': 0.7, 'bagging_freq': 50, 'feature_fraction': 0.7, 'num_leaves': 80}",0.263676,0.282242,0.296250,0.150671,0.248211,0.057486,9
1,0.004186,0.7,50,0.7,120,"{'bagging_fraction': 0.7, 'bagging_freq': 50, 'feature_fraction': 0.7, 'num_leaves': 120}",0.263795,0.281187,0.296068,0.149960,0.247754,0.057604,10


In [16]:
write_submission(model, X_test)

In [17]:
tc_len = len(tree_cols)
print(dict(zip(tree_cols, model.best_estimator_.feature_importances_[:tc_len]) ))

{'match_product_title': 223, 'match_descr': 155, 'len_search_term_words': 244, 'len_search_term_letters': 637, 'match_pos_title': 162, 'match_pos_descr': 137, 'prod_title_ind': 595, 'descr_ind': 504, 'words_std_title': 251, 'words_std_descr': 239, 'len_title': 423, 'len_descr': 381, 'last_word_title': 98, 'last_word_descr': 93, 'match_orig_title': 216, 'match_orig_descr': 160, 'fuzzy_title': 640, 'fuzzy_descr': 398, 'fuzzy_orig_title': 544, 'fuzzy_orig_descr': 338, 'part_fuzzy_title': 179, 'part_fuzzy_descr': 129, 'part_fuzzy_orig_title': 209, 'part_fuzzy_orig_descr': 101, 'first_word_title': 85, 'first_word_descr': 52, 'match_numbers_title': 155, 'fuzzy_brand': 89}


In [21]:
encoding = "ISO-8859-1"
attrs = pd.read_csv('zip/attributes.csv.gz', encoding=encoding, compression='gzip', index_col=['product_uid'])

/home/eugene/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [26]:
attrs['name'].value_counts()[:30]

MFG Brand Name                 86250
Bullet02                       86248
Bullet03                       86226
Bullet04                       86174
Bullet01                       85940
Product Width (in.)            61137
Bullet05                       60529
Product Height (in.)           54698
Product Depth (in.)            53652
Product Weight (lb.)           45175
Bullet06                       44901
Color Family                   41508
Bullet07                       34349
Material                       31500
Color/Finish                   28564
Bullet08                       26645
Certifications and Listings    24583
Bullet09                       20567
Assembled Height (in.)         18299
Assembled Width (in.)          18263
Assembled Depth (in.)          18198
Product Length (in.)           16705
Bullet10                       14763
Indoor/Outdoor                 12939
Bullet11                       11784
Commercial / Residential       9530 
Bullet12                       8795 
E

In [41]:
colors = attrs[attrs['name'] == 'Color Family']['value'].groupby(['product_uid']).max()
colors

product_uid
100002.0    Browns / Tans    
100003.0    White            
100005.0    Chrome           
100009.0    White            
100010.0    Metallic         
100012.0    Tan              
100013.0    Black            
100015.0    Tan              
100016.0    Blacks           
100018.0    White            
100019.0    White            
100021.0    Blacks           
100023.0    Gray             
100026.0    Tan              
100028.0    Tan              
100030.0    White            
100031.0    Assorted Colors  
100032.0    Gray             
100035.0    Metallics        
100036.0    White            
100037.0    Black            
100038.0    Orange           
100040.0    White            
100042.0    Tan              
100043.0    Gray             
100044.0    White            
100046.0    Green            
100049.0    White            
100052.0    Beige            
100054.0    White            
            ...              
224366.0    Browns / Tans    
224367.0    Reds / Pinks    

In [36]:
bullets_names = ['Bullet0' + str(i) for i in range(1, 10)]
bullets = attrs[attrs['name'].isin(bullets_names)]
bullets = bullets.groupby(['product_uid'])['value'].apply(lambda x: ' '.join([str(a) for a in x]))
bullets

product_uid
100001.0    Versatile connector for various 90Â° connections and home repair projects Stronger than angled nailing or screw fastening alone Help ensure joints are consistently straight and strong Dimensions: 3 in. x 3 in. x 1-1/2 in. Made from 12-Gauge steel Galvanized for extra corrosion resistance Install with 10d common nails or #9 x 1-1/2 in. Strong-Drive SD screws                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [46]:
material = attrs[attrs['name'] == 'Material']['value'].groupby(['product_uid']).max()
material

product_uid
100001.0    Galvanized Steel               
100003.0    Composite                      
100007.0    Plastic                        
100009.0    Medium Density Fiberboard (MDF)
100010.0    Steel                          
100012.0    Bamboo                         
100014.0    Plastic                        
100015.0    Bamboo                         
100017.0    Aluminum                       
100018.0    Plastic                        
100019.0    Medium Density Fiberboard (MDF)
100023.0    Concrete                       
100026.0    Concrete                       
100028.0    Wood                           
100030.0    Steel                          
100032.0    PVC                            
100033.0    Wire                           
100034.0    Tampico                        
100036.0    Metal                          
100037.0    Fabric                         
100039.0    Steel                          
100040.0    Vinyl                          
100042.0    Vitreous

In [38]:
stemmer = SnowballStemmer('english')
stop_words = set(stopwords.words('english'))


def replaces(s):
    if isinstance(s, str):
        s = s.lower()
        s = re.sub(r"(\w)\.([A-Z])", r"\1 \2", s) 
    
        s = s.replace(" x "," xby ")
        s = s.replace("*"," xby ")
        s = s.replace(" by "," xby")
        s = s.replace("x0"," xby 0")
        s = s.replace("x1"," xby 1")
        s = s.replace("x2"," xby 2")
        s = s.replace("x3"," xby 3")
        s = s.replace("x4"," xby 4")
        s = s.replace("x5"," xby 5")
        s = s.replace("x6"," xby 6")
        s = s.replace("x7"," xby 7")
        s = s.replace("x8"," xby 8")
        s = s.replace("x9"," xby 9")
        s = s.replace("0x","0 xby ")
        s = s.replace("1x","1 xby ")
        s = s.replace("2x","2 xby ")
        s = s.replace("3x","3 xby ")
        s = s.replace("4x","4 xby ")
        s = s.replace("5x","5 xby ")
        s = s.replace("6x","6 xby ")
        s = s.replace("7x","7 xby ")
        s = s.replace("8x","8 xby ")
        s = s.replace("9x","9 xby ")
        
        s = re.sub(r"([0-9])( *)\.( *)([0-9])", r"\1.\4", s)
        s = re.sub(r"([0-9]+)( *)(inches|inch|in|')\.?", r"\1in. ", s)
        s = re.sub(r"([0-9]+)( *)(foot|feet|ft|'')\.?", r"\1ft. ", s)
        s = re.sub(r"([0-9]+)( *)(pounds|pound|lbs|lb)\.?", r"\1lb. ", s)
        s = re.sub(r"([0-9]+)( *)(square|sq) ?\.?(feet|foot|ft)\.?", r"\1sq.ft. ", s)
        s = re.sub(r"([0-9]+)( *)(cubic|cu) ?\.?(feet|foot|ft)\.?", r"\1cu.ft. ", s)
        s = re.sub(r"([0-9]+)( *)(gallons|gallon|gal)\.?", r"\1gal. ", s)
        s = re.sub(r"([0-9]+)( *)(ounces|ounce|oz)\.?", r"\1oz. ", s)
        s = re.sub(r"([0-9]+)( *)(centimeters|cm)\.?", r"\1cm. ", s)
        s = re.sub(r"([0-9]+)( *)(milimeters|mm)\.?", r"\1mm. ", s)
        s = s.replace("°"," degrees ")
        s = re.sub(r"([0-9]+)( *)(degrees|degree)\.?", r"\1deg. ", s)
        s = s.replace(" v "," volts ")
        s = re.sub(r"([0-9]+)( *)(volts|volt)\.?", r"\1volt. ", s)
        s = re.sub(r"([0-9]+)( *)(watts|watt)\.?", r"\1watt. ", s)
        s = re.sub(r"([0-9]+)( *)(amperes|ampere|amps|amp)\.?", r"\1amp. ", s)
        
        s = s.replace("toliet","toilet")
        s = s.replace("airconditioner","air conditioner")
        s = s.replace("vinal","vinyl")
        s = s.replace("vynal","vinyl")
        s = s.replace("skill","skil")
        s = s.replace("snowbl","snow bl")
        s = s.replace("plexigla","plexi gla")
        s = s.replace("rustoleum","rust-oleum")
        s = s.replace("whirpool","whirlpool")
        s = s.replace("whirlpoolga", "whirlpool ga")
        s = s.replace("whirlpoolstainless","whirlpool stainless")

        s = s.replace("  "," ")
        
        s = s.replace("/", " ")
        s = s.replace("-", " ")
        return " ".join([re.sub('[^A-Za-z0-9./]', ' ', word)
                         for word in s.split()])
    else:
        return "null"

@np.vectorize
def stem_sentence(s): 
    s = replaces(s)   
    return str(" ".join([stemmer.stem(word)
                         for word in s.split()
                         if word not in stop_words]))


In [42]:
colors = colors.apply(stem_sentence)

In [ ]:
bullets = bullets.apply(stem_sentence)

In [47]:
material = material.apply(stem_sentence)

In [50]:
train['color'] = train.product_uid.map(colors).astype(str)
train['bullets'] = train.product_uid.map(bullets).astype(str)
train['material'] = train.product_uid.map(material).astype(str)
train['search_term'] = train['search_term'].astype(str)

train['match_color'] = word_match_count(train['color'], train['search_term'])
train['match_bullets'] = word_match_count(train['bullets'], train['search_term'])
train['match_material'] = word_match_count(train['material'], train['search_term'])

train.head()

,product_uid,product_title,search_term,relevance,orig_search_term,orig_product_title,descr,orig_descr,match_product_title,match_descr,len_search_term_words,len_search_term_letters,complete_match_title,complete_match_descr,brand,match_brand,words_std_title,words_std_descr,last_word_title,last_word_descr,prod_title_ind,descr_ind,match_pos_title,match_pos_descr,match_orig_title,match_orig_descr,len_title,len_descr,fuzzy_title,fuzzy_descr,fuzzy_orig_title,fuzzy_orig_descr,part_fuzzy_title,part_fuzzy_descr,part_fuzzy_orig_title,part_fuzzy_orig_descr,pred_diff,first_word_title,first_word_descr,match_numbers_title,match_numbers_descr,fuzzy_brand,color,bullets,material,match_color,match_bullets,match_material
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,100001,simpson strong tie 12 gaug angl,angl bracket,3.00,angle bracket,Simpson Strong-Tie 12-Gauge Angle,"not angl make joint stronger, also provid consistent, straight corners. simpson strong-ti offer wide varieti angl various size thick handl light-duti job project structur connect needed. some bent (skewed) match project. for outdoor project moistur present, use zmax zinc-coat connectors, provid extra resist corros (look ""z"" end model number).versatil connector various 90 connect home repair projectsstrong angl nail screw fasten alonehelp ensur joint consist straight strongdimensions: 3 in. x 3 in. x 1-1/2 in.mad 12-gaug steelgalvan extra corros resistanceinstal 10d common nail #9 x 1-1/2 in. strong-driv sd screw","Not only do angles make joints stronger, they also provide more consistent, straight corners. Simpson Strong-Tie offers a wide variety of angles in various sizes and thicknesses to handle light-duty jobs or projects where a structural connection is needed. Some can be bent (skewed) to match the project. For outdoor projects or those where moisture is present, use our ZMAX zinc-coated connectors, which provide extra resistance against corrosion (look for a ""Z"" at the end of the model number).Versatile connector for various 90 connections and home repair projectsStronger than angled nailing or screw fastening aloneHelp ensure joints are consistently straight and strongDimensions: 3 in. x 3 in. x 1-1/2 in.Made from 12-Gauge steelGalvanized for extra corrosion resistanceInstall with 10d common nails or #9 x 1-1/2 in. Strong-Drive SD screws",0.500000,0.500000,2,12,0,0,simpson strong-ti,0.0,0.00,0.00,0,0,0.833333,0.011236,0.500000,0.000000,0.500000,0.000000,6,89,50,50,56,1,100,100,100,62,0.167173,1,1,0,0,0,nan,versatil connector various 90 degre connect home repair project stronger angl nail screw fasten alon help ensur joint consist straight strong dimens 3in. xbi 3in. xbi 1 1 2in. made 12 gaug steel galvan extra corros resist instal 10d common nail 9 xbi 1 1 2in. strong drive sd screw,galvan steel,0.0,0.020000,0.0
3,100001,simpson strong tie 12 gaug angl,l bracket,2.50,l bracket,Simpson Strong-Tie 12-Gauge Angle,"not angl make joint stronger, also provid consistent, straight corners. simpson strong-ti offer wide varieti angl various size thick handl light-duti job project structur connect needed. some bent (skewed) match project. for outdoor project moistur present, use zmax zinc-coat connectors, provid extra resist corros (look ""z"" end model number).versatil connector various 90 connect home repair projectsstrong angl nail screw fasten alonehelp ensur joint consist straight strongdimensions: 3 in. x 3 in. x 1-1/2 in.mad 12-gaug steelgalvan extra corros resistanceinstal 10d common nail #9 x 1-1/2 in. strong-driv sd screw","Not only do angles make joints stronger, they also provide more consistent, straight corners. Simpson Strong-Tie offers a wide variety of angles in various sizes and thicknesses to handle light-duty jobs or projects where a structural connection is needed. Some can be bent (skewed) to match the project. For outdoor projects or those where moisture is present, use our ZMAX zinc-coated connectors, which provide extra resistance against corrosion

In [51]:
train.to_csv('zip/train6.csv.gz', compression='gzip')

In [52]:
test['color'] = test.product_uid.map(colors).astype(str)
test['bullets'] = test.product_uid.map(bullets).astype(str)
test['material'] = test.product_uid.map(material).astype(str)
test['search_term'] = test['search_term'].astype(str)

test['match_color'] = word_match_count(test['color'], test['search_term'])
test['match_bullets'] = word_match_count(test['bullets'], test['search_term'])
test['match_material'] = word_match_count(test['material'], test['search_term'])

test.head()

,product_uid,product_title,search_term,orig_search_term,orig_product_title,descr,orig_descr,match_product_title,match_descr,len_search_term_words,len_search_term_letters,complete_match_title,complete_match_descr,brand,match_brand,words_std_title,words_std_descr,last_word_title,last_word_descr,match_pos_title,match_pos_descr,match_orig_title,match_orig_descr,len_title,len_descr,prod_title_ind,descr_ind,fuzzy_title,fuzzy_descr,fuzzy_orig_title,fuzzy_orig_descr,part_fuzzy_title,part_fuzzy_descr,part_fuzzy_orig_title,part_fuzzy_orig_descr,first_word_title,first_word_descr,match_numbers_title,match_numbers_descr,fuzzy_brand,color,bullets,material,match_color,match_bullets,match_material
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,100001,simpson strong tie 12 gaug angl,90deg. bracket,90 degree bracket,Simpson Strong-Tie 12-Gauge Angle,"not angl make joint stronger, also provid consistent, straight corners. simpson strong-ti offer wide varieti angl various size thick handl light-duti job project structur connect needed. some bent (skewed) match project. for outdoor project moistur present, use zmax zinc-coat connectors, provid extra resist corros (look ""z"" end model number).versatil connector various 90 connect home repair projectsstrong angl nail screw fasten alonehelp ensur joint consist straight strongdimensions: 3 in. x 3 in. x 1-1/2 in.mad 12-gaug steelgalvan extra corros resistanceinstal 10d common nail #9 x 1-1/2 in. strong-driv sd screw","Not only do angles make joints stronger, they also provide more consistent, straight corners. Simpson Strong-Tie offers a wide variety of angles in various sizes and thicknesses to handle light-duty jobs or projects where a structural connection is needed. Some can be bent (skewed) to match the project. For outdoor projects or those where moisture is present, use our ZMAX zinc-coated connectors, which provide extra resistance against corrosion (look for a ""Z"" at the end of the model number).Versatile connector for various 90 connections and home repair projectsStronger than angled nailing or screw fastening aloneHelp ensure joints are consistently straight and strongDimensions: 3 in. x 3 in. x 1-1/2 in.Made from 12-Gauge steelGalvanized for extra corrosion resistanceInstall with 10d common nails or #9 x 1-1/2 in. Strong-Drive SD screws",0.000000,0.000000,2,14,0,0,simpson strong-ti,0.0,0,0,0,0,0.000000,0.000000,0.000000,0.333333,6,89,1.0,1.00000,18,2,12,21,23,38,35,100,0,0,0.0,1.0,0,nan,versatil connector various 90 degre connect home repair project stronger angl nail screw fasten alon help ensur joint consist straight strong dimens 3in. xbi 3in. xbi 1 1 2in. made 12 gaug steel galvan extra corros resist instal 10d common nail 9 xbi 1 1 2in. strong drive sd screw,galvan steel,0.0,0.00,0.0
4,100001,simpson strong tie 12 gaug angl,metal l bracket,metal l brackets,Simpson Strong-Tie 12-Gauge Angle,"not angl make joint stronger, also provid consistent, straight corners. simpson strong-ti offer wide varieti angl various size thick handl light-duti job project structur connect needed. some bent (skewed) match project. for outdoor project moistur present, use zmax zinc-coat connectors, provid extra resist corros (look ""z"" end model number).versatil connector various 90 connect home repair projectsstrong angl nail screw fasten alonehelp ensur joint consist straight strongdimensions: 3 in. x 3 in. x 1-1/2 in.mad 12-gaug steelgalvan extra corros resistanceinstal 10d common nail #9 x 1-1/2 in. strong-driv sd screw","Not only do angles make joints stronger, they also provide more consistent, straight corners. Simpson Strong-Tie offers a wide variety of angles in various sizes and thicknesses to handle light-duty jobs or projects where a structural connection is needed. Some can be bent (skewed) to match the project. For outdoor projects or those where moisture is present, use our ZMAX zinc-coated connectors, which provide extra resistance against corrosion (look for a ""Z"" at the end of the

In [53]:
test.to_csv('zip/test6.csv.gz', compression='gzip')

In [54]:
tree_cols = ['match_product_title', 'match_descr', 'len_search_term_words', 'len_search_term_letters',
             'match_pos_title', 'match_pos_descr', 'prod_title_ind', 'descr_ind',
             'words_std_title', 'words_std_descr', 'len_title', 'len_descr', 'last_word_title',
             'last_word_descr', 'match_orig_title', 'match_orig_descr',
             'fuzzy_title', 'fuzzy_descr', 'fuzzy_orig_title', 'fuzzy_orig_descr',
             'part_fuzzy_title', 'part_fuzzy_descr', 'part_fuzzy_orig_title', 'part_fuzzy_orig_descr',
             'first_word_title', 'first_word_descr',
             'match_numbers_title',
             'fuzzy_brand',
             'match_color', 'match_bullets', 'match_material']

use_cols = tree_cols + ['search_term', 'product_title', 'descr', 'brand']

clf = fitter()
X_train = clf.transform(train[use_cols])
X_test = clf.transform(test[use_cols])
y = train['relevance'].values

gbm = lgb.LGBMRegressor(learning_rate=0.03, colsample_bytree=0.5, num_leaves=80, n_estimators=1000)
gbm.fit(X_train, y)

tc_len = len(tree_cols)
print(dict(zip(tree_cols, gbm.feature_importances_[:tc_len]) ))

write_submission(gbm, X_test)

{'match_product_title': 261, 'match_descr': 159, 'len_search_term_words': 168, 'len_search_term_letters': 646, 'match_pos_title': 166, 'match_pos_descr': 152, 'prod_title_ind': 566, 'descr_ind': 462, 'words_std_title': 246, 'words_std_descr': 235, 'len_title': 392, 'len_descr': 367, 'last_word_title': 90, 'last_word_descr': 70, 'match_orig_title': 201, 'match_orig_descr': 151, 'fuzzy_title': 557, 'fuzzy_descr': 385, 'fuzzy_orig_title': 503, 'fuzzy_orig_descr': 355, 'part_fuzzy_title': 169, 'part_fuzzy_descr': 116, 'part_fuzzy_orig_title': 197, 'part_fuzzy_orig_descr': 98, 'first_word_title': 91, 'first_word_descr': 60, 'match_numbers_title': 147, 'fuzzy_brand': 97, 'match_color': 4, 'match_bullets': 239, 'match_material': 9}


0.46846

In [55]:
gbm.score(X_train, y)

0.6510125559714324

In [56]:
tree_cols = ['match_product_title', 'match_descr', 'len_search_term_words', 'len_search_term_letters',
             'match_pos_title', 'match_pos_descr', 'prod_title_ind', 'descr_ind',
             'words_std_title', 'words_std_descr', 'len_title', 'len_descr', 'last_word_title',
             'last_word_descr', 'match_orig_title', 'match_orig_descr',
             'fuzzy_title', 'fuzzy_descr', 'fuzzy_orig_title', 'fuzzy_orig_descr',
             'part_fuzzy_title', 'part_fuzzy_descr', 'part_fuzzy_orig_title', 'part_fuzzy_orig_descr',
             'first_word_title', 'first_word_descr',
             'match_numbers_title',
             'fuzzy_brand',
             'match_bullets']

use_cols = tree_cols + ['search_term', 'product_title', 'descr', 'brand']

clf = fitter()
X_train = clf.transform(train[use_cols])
X_test = clf.transform(test[use_cols])

gbm = lgb.LGBMRegressor(learning_rate=0.03, colsample_bytree=0.5, num_leaves=80, n_estimators=1000)
gbm.fit(X_train, y)

tc_len = len(tree_cols)
print(dict(zip(tree_cols, gbm.feature_importances_[:tc_len]) ))

write_submission(gbm, X_test)

{'match_product_title': 249, 'match_descr': 178, 'len_search_term_words': 229, 'len_search_term_letters': 631, 'match_pos_title': 145, 'match_pos_descr': 140, 'prod_title_ind': 567, 'descr_ind': 438, 'words_std_title': 219, 'words_std_descr': 249, 'len_title': 390, 'len_descr': 349, 'last_word_title': 90, 'last_word_descr': 85, 'match_orig_title': 220, 'match_orig_descr': 160, 'fuzzy_title': 535, 'fuzzy_descr': 348, 'fuzzy_orig_title': 485, 'fuzzy_orig_descr': 294, 'part_fuzzy_title': 192, 'part_fuzzy_descr': 130, 'part_fuzzy_orig_title': 214, 'part_fuzzy_orig_descr': 101, 'first_word_title': 82, 'first_word_descr': 63, 'match_numbers_title': 134, 'fuzzy_brand': 95, 'match_bullets': 181}


0.46775

In [60]:
train['pred_diff'] = train['relevance'] - gbm.predict(X_train)
train.sort_values(['pred_diff'], ascending=False)[:50]

,product_uid,product_title,search_term,relevance,orig_search_term,orig_product_title,descr,orig_descr,match_product_title,match_descr,len_search_term_words,len_search_term_letters,complete_match_title,complete_match_descr,brand,match_brand,words_std_title,words_std_descr,last_word_title,last_word_descr,prod_title_ind,descr_ind,match_pos_title,match_pos_descr,match_orig_title,match_orig_descr,len_title,len_descr,fuzzy_title,fuzzy_descr,fuzzy_orig_title,fuzzy_orig_descr,part_fuzzy_title,part_fuzzy_descr,part_fuzzy_orig_title,part_fuzzy_orig_descr,pred_diff,first_word_title,first_word_descr,match_numbers_title,match_numbers_descr,fuzzy_brand,color,bullets,material,match_color,match_bullets,match_material
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
203914,192088,behr premium plus 600e 2 harbor mist zero voc interior paint,locit 2 plus 1,3.00,locite 2 plus 1,BEHR Premium Plus #600E-2 Harbor Mist Zero VOC Interior Paint,"behr premium plus zero voc, self-prim interior flat 100% acryl provid mildew resist finish. this easy-clean flat provid excel touch-up, ultim durabl except hide. the finish display non-reflective, matt appearance.california residents: see&nbsp;proposit 65 informationid famili rooms, live rooms, dine rooms, bedroom ceilings100% acryl finish easi cleaningtough durabl matt finishexcept hide, 1 cover 400 sq. ft. depend color select surfac porosityeasi soap water clean-upexcept latex formula provid mildew resist finishself prime finish seal surfaceactu paint color may vari on-screen printer representationsonlin price includ paint care fee follow states: ca, co, ct, me, mn, or, ri, vt","BEHR Premium Plus Zero VOC, Self-Priming Interior Flat is 100% acrylic and provides a mildew resistant finish. This easy-clean flat provides excellent touch-up, ultimate durability and exceptional hide. The finish displays a non-reflective, matte appearance.California residents: see&nbsp;Proposition 65 informationIdeal for family rooms, living rooms, dining rooms, bedrooms and ceilings100% acrylic finish for easy cleaningTough and durable matte finishExceptional hide, 1 can covers up to 400 sq. ft. depending on color selected and surface porosityEasy soap and water clean-upExceptional latex formula provides a mildew resistant finishSelf priming finish that seals the surfaceActual paint colors may vary from on-screen and printer representationsOnline Price includes Paint Care fee in the following states: CA, CO, CT, ME, MN, OR, RI, VT",0.500000,0.500000,4,14,0,0,behr premium plus,0.333333,1.00,576.00,0,1,0.181818,0.021053,0.500000,0.500000,0.250000,0.500000,11,95,60,60,57,57,100,100,100,100,1.110743,0,0,0,0,100,purpl lavend,ideal famili room live room dine room bedroom ceil 100 acryl finish easi clean tough durabl matt finish except hide 1 cover 400 sq. ft. depend color select surfac poros easi soap water clean except latex formula provid mildew resist finish self prime finish seal surfac actual paint color may vari screen printer represent,nan,0.0,0.018519,0.000000
194147,184559,lenmar nickel metal hydrid 1200mah 3.6 volt cordless phone replac batteri,polish batteri metal,3.00,polisher battery metal,Lenmar Nickel-Metal Hydride 1200mAh/3.6-Volt Cordless Phone Replacement Battery,"power panasonic, american telecom, tele-phone, key phone, audiovox, uniden, universal, zeta cordless phone lenmar nickel-met hydrid cordless phone replac battery. this 3.6-volt/1200mah batteri provid reliable, long-last power. compat panason kx-a150, american telecom 2110, tele-phon tel-1000, key phone 920-wd, audiovox at-12a, uniden ex-3300, univers 2000, zeta 510.fits: panason kx-a150, american telecom 2110, tele-phon tel-1000, key phone 920-wd, audiovox at-12a, uniden ex-3300, univers 2000, zeta 510replaces: dantona 3aa, empir cpb-400, general electr bt-17, panason p-p341pa, recoton t121, uniden ba-3003.6-volt / 1200mahnickel-met hydride1 year warranti","Power your Panasonic, American Telecom, Tele-Phone, Key Phone, Audiovox, Uniden, Universal, and Zeta c

In [62]:
def fitter():
    clf = Pipeline([
    ('union', FeatureUnion(
                transformer_list = [
                    ('cst',  cust_regression_vals()),  
                    ('txt1', Pipeline([('s1', cust_txt_col(col='search_term')),
                                       ('tfidf1', TfidfVectorizer(stop_words='english')),
                                       ('tsvd1', TruncatedSVD(n_components=40))])),
                    ('txt2', Pipeline([('s2', cust_txt_col(col='product_title')),
                                       ('tfidf2', TfidfVectorizer(stop_words='english')),
                                       ('tsvd2', TruncatedSVD(n_components=40))])),
                    ('txt3', Pipeline([('s3', cust_txt_col(col='descr')),
                                       ('tfidf3', TfidfVectorizer(stop_words='english')),
                                       ('tsvd3', TruncatedSVD(n_components=30))])),
                    ('txt4', Pipeline([('s4', cust_txt_col(col='brand')),
                                       ('tfidf4', TfidfVectorizer(stop_words='english')),
                                       ('tsvd4', TruncatedSVD(n_components=15))])),
                    ('txt5', Pipeline([('s5', cust_txt_col(col='bullets')),
                                       ('tfidf5', TfidfVectorizer(stop_words='english')),
                                       ('tsvd5', TruncatedSVD(n_components=30))]))
                    ]
            ))])
    clf.fit(train[use_cols])
    return clf

use_cols = tree_cols + ['search_term', 'product_title', 'descr', 'brand', 'bullets']

clf = fitter()
X_train = clf.transform(train[use_cols])
X_test = clf.transform(test[use_cols])

gbm = lgb.LGBMRegressor(learning_rate=0.03, colsample_bytree=0.5, num_leaves=80, n_estimators=1000)
gbm.fit(X_train, y)

tc_len = len(tree_cols)
print(dict(zip(tree_cols, gbm.feature_importances_[:tc_len]) ))

write_submission(gbm, X_test)

{'match_product_title': 315, 'match_descr': 218, 'len_search_term_words': 284, 'len_search_term_letters': 821, 'match_pos_title': 252, 'match_pos_descr': 212, 'prod_title_ind': 784, 'descr_ind': 679, 'words_std_title': 307, 'words_std_descr': 337, 'len_title': 500, 'len_descr': 581, 'last_word_title': 124, 'last_word_descr': 100, 'match_orig_title': 293, 'match_orig_descr': 218, 'fuzzy_title': 755, 'fuzzy_descr': 565, 'fuzzy_orig_title': 691, 'fuzzy_orig_descr': 539, 'part_fuzzy_title': 204, 'part_fuzzy_descr': 170, 'part_fuzzy_orig_title': 235, 'part_fuzzy_orig_descr': 102, 'first_word_title': 127, 'first_word_descr': 75, 'match_numbers_title': 175, 'fuzzy_brand': 109, 'match_bullets': 291}


In [63]:
print(gbm.feature_importances_[tc_len:])

[627 649 588 587 557 542 540 493 459 453 491 492 497 535 549 476 544 429
 509 534 443 526 497 463 483 567 603 521 496 556 549 614 626 570 689 670
 650 575 575 594 633 455 420 468 477 422 412 378 479 500 486 519 435 487
 487 478 534 451 457 559 446 522 453 484 440 459 380 504 434 499 522 475
 473 520 551 488 538 519 502 469 592 452 416 460 471 373 447 494 416 478
 418 461 518 429 411 417 459 464 523 454 482 492 475 470 521 459 484 481
 527 501 301 241 206 227 303 202 220 208 226 201 216 206 184 218 236 423
 358 346 278 333 306 349 350 377 295 316 348 313 377 333 336 347 307 321
 345 376 348 355 344 388 377 369 411 372 366]


In [64]:
def fitter():
    clf = Pipeline([
    ('union', FeatureUnion(
                transformer_list = [
                    ('cst',  cust_regression_vals()),  
                    ('txt1', Pipeline([('s1', cust_txt_col(col='search_term')),
                                       ('tfidf1', TfidfVectorizer(stop_words='english')),
                                       ('tsvd1', TruncatedSVD(n_components=80))])),
                    ('txt2', Pipeline([('s2', cust_txt_col(col='product_title')),
                                       ('tfidf2', TfidfVectorizer(stop_words='english')),
                                       ('tsvd2', TruncatedSVD(n_components=80))])),
                    ('txt3', Pipeline([('s3', cust_txt_col(col='descr')),
                                       ('tfidf3', TfidfVectorizer(stop_words='english')),
                                       ('tsvd3', TruncatedSVD(n_components=60))])),
                    ('txt4', Pipeline([('s4', cust_txt_col(col='brand')),
                                       ('tfidf4', TfidfVectorizer(stop_words='english')),
                                       ('tsvd4', TruncatedSVD(n_components=30))])),
                    ('txt5', Pipeline([('s5', cust_txt_col(col='bullets')),
                                       ('tfidf5', TfidfVectorizer(stop_words='english')),
                                       ('tsvd5', TruncatedSVD(n_components=15))]))
                    ]
            ))])
    clf.fit(train[use_cols])
    return clf

clf = fitter()
X_train = clf.transform(train[use_cols])
X_test = clf.transform(test[use_cols])

gbm = lgb.LGBMRegressor(learning_rate=0.03, colsample_bytree=0.5, num_leaves=80, n_estimators=1000)
gbm.fit(X_train, y)

tc_len = len(tree_cols)
print(dict(zip(tree_cols, gbm.feature_importances_[:tc_len]) ))
print(gbm.feature_importances_[tc_len:])

write_submission(gbm, X_test)

{'match_product_title': 248, 'match_descr': 159, 'len_search_term_words': 221, 'len_search_term_letters': 596, 'match_pos_title': 143, 'match_pos_descr': 111, 'prod_title_ind': 565, 'descr_ind': 438, 'words_std_title': 254, 'words_std_descr': 245, 'len_title': 362, 'len_descr': 342, 'last_word_title': 84, 'last_word_descr': 67, 'match_orig_title': 228, 'match_orig_descr': 173, 'fuzzy_title': 557, 'fuzzy_descr': 399, 'fuzzy_orig_title': 464, 'fuzzy_orig_descr': 349, 'part_fuzzy_title': 167, 'part_fuzzy_descr': 134, 'part_fuzzy_orig_title': 203, 'part_fuzzy_orig_descr': 94, 'first_word_title': 92, 'first_word_descr': 62, 'match_numbers_title': 141, 'fuzzy_brand': 103, 'match_bullets': 197}
[425 408 369 349 311 295 278 291 264 270 276 259 251 304 336 303 303 258
 255 302 252 261 283 262 297 266 382 261 316 296 268 283 328 293 291 290
 272 344 334 324 314 256 239 280 252 317 294 249 309 271 269 318 294 266
 267 326 321 292 296 320 304 334 322 302 298 313 354 315 314 409 335 360
 347 362 34

In [66]:
train['fuzzy_bullets'] = fuzzy_match_partial(train['orig_search_term'], train['bullets'])
test['fuzzy_bullets'] = fuzzy_match_partial(test['orig_search_term'], test['bullets'])

tree_cols = ['match_product_title', 'match_descr', 'len_search_term_words', 'len_search_term_letters',
             'match_pos_title', 'match_pos_descr', 'prod_title_ind', 'descr_ind',
             'words_std_title', 'words_std_descr', 'len_title', 'len_descr', 'last_word_title',
             'last_word_descr', 'match_orig_title', 'match_orig_descr',
             'fuzzy_title', 'fuzzy_descr', 'fuzzy_orig_title', 'fuzzy_orig_descr',
             'part_fuzzy_title', 'part_fuzzy_descr', 'part_fuzzy_orig_title', 'part_fuzzy_orig_descr',
             'first_word_title', 'first_word_descr',
             'match_numbers_title',
             'fuzzy_brand',
             'match_bullets', 'fuzzy_bullets']

use_cols = tree_cols + ['search_term', 'product_title', 'descr', 'brand', 'bullets']

clf = fitter()
X_train = clf.transform(train[use_cols])
X_test = clf.transform(test[use_cols])

gbm = lgb.LGBMRegressor(learning_rate=0.03, colsample_bytree=0.5, num_leaves=80, n_estimators=1000)
gbm.fit(X_train, y)

tc_len = len(tree_cols)
print(dict(zip(tree_cols, gbm.feature_importances_[:tc_len]) ))
print(gbm.feature_importances_[tc_len:])

write_submission(gbm, X_test)

{'match_product_title': 246, 'match_descr': 171, 'len_search_term_words': 218, 'len_search_term_letters': 642, 'match_pos_title': 157, 'match_pos_descr': 146, 'prod_title_ind': 577, 'descr_ind': 448, 'words_std_title': 274, 'words_std_descr': 237, 'len_title': 368, 'len_descr': 312, 'last_word_title': 86, 'last_word_descr': 66, 'match_orig_title': 212, 'match_orig_descr': 166, 'fuzzy_title': 567, 'fuzzy_descr': 375, 'fuzzy_orig_title': 470, 'fuzzy_orig_descr': 297, 'part_fuzzy_title': 175, 'part_fuzzy_descr': 122, 'part_fuzzy_orig_title': 205, 'part_fuzzy_orig_descr': 82, 'first_word_title': 88, 'first_word_descr': 65, 'match_numbers_title': 155, 'fuzzy_brand': 82, 'match_bullets': 193, 'fuzzy_bullets': 193}
[408 408 323 320 320 346 260 267 246 276 296 285 269 324 332 304 298 272
 244 249 280 302 309 315 298 278 367 251 278 310 288 230 360 297 270 297
 295 301 332 314 298 302 302 294 309 307 285 289 318 305 286 285 350 286
 288 337 357 277 307 320 332 330 297 295 323 372 310 290 322 34

In [67]:
train['pred_diff'] = train['relevance'] - gbm.predict(X_train)
train.sort_values(['pred_diff'], ascending=False)[:50]

,product_uid,product_title,search_term,relevance,orig_search_term,orig_product_title,descr,orig_descr,match_product_title,match_descr,len_search_term_words,len_search_term_letters,complete_match_title,complete_match_descr,brand,match_brand,words_std_title,words_std_descr,last_word_title,last_word_descr,prod_title_ind,descr_ind,match_pos_title,match_pos_descr,match_orig_title,match_orig_descr,len_title,len_descr,fuzzy_title,fuzzy_descr,fuzzy_orig_title,fuzzy_orig_descr,part_fuzzy_title,part_fuzzy_descr,part_fuzzy_orig_title,part_fuzzy_orig_descr,pred_diff,first_word_title,first_word_descr,match_numbers_title,match_numbers_descr,fuzzy_brand,color,bullets,material,match_color,match_bullets,match_material,fuzzy_bullets
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
203914,192088,behr premium plus 600e 2 harbor mist zero voc interior paint,locit 2 plus 1,3.00,locite 2 plus 1,BEHR Premium Plus #600E-2 Harbor Mist Zero VOC Interior Paint,"behr premium plus zero voc, self-prim interior flat 100% acryl provid mildew resist finish. this easy-clean flat provid excel touch-up, ultim durabl except hide. the finish display non-reflective, matt appearance.california residents: see&nbsp;proposit 65 informationid famili rooms, live rooms, dine rooms, bedroom ceilings100% acryl finish easi cleaningtough durabl matt finishexcept hide, 1 cover 400 sq. ft. depend color select surfac porosityeasi soap water clean-upexcept latex formula provid mildew resist finishself prime finish seal surfaceactu paint color may vari on-screen printer representationsonlin price includ paint care fee follow states: ca, co, ct, me, mn, or, ri, vt","BEHR Premium Plus Zero VOC, Self-Priming Interior Flat is 100% acrylic and provides a mildew resistant finish. This easy-clean flat provides excellent touch-up, ultimate durability and exceptional hide. The finish displays a non-reflective, matte appearance.California residents: see&nbsp;Proposition 65 informationIdeal for family rooms, living rooms, dining rooms, bedrooms and ceilings100% acrylic finish for easy cleaningTough and durable matte finishExceptional hide, 1 can covers up to 400 sq. ft. depending on color selected and surface porosityEasy soap and water clean-upExceptional latex formula provides a mildew resistant finishSelf priming finish that seals the surfaceActual paint colors may vary from on-screen and printer representationsOnline Price includes Paint Care fee in the following states: CA, CO, CT, ME, MN, OR, RI, VT",0.500000,0.500000,4,14,0,0,behr premium plus,0.333333,1.00,576.00,0,1,0.181818,0.021053,0.500000,0.500000,0.250000,0.500000,11,95,60,60,57,57,100,100,100,100,1.155456,0,0,0,0,100,purpl lavend,ideal famili room live room dine room bedroom ceil 100 acryl finish easi clean tough durabl matt finish except hide 1 cover 400 sq. ft. depend color select surfac poros easi soap water clean except latex formula provid mildew resist finish self prime finish seal surfac actual paint color may vari screen printer represent,nan,0.0,0.018519,0.000000,100
194147,184559,lenmar nickel metal hydrid 1200mah 3.6 volt cordless phone replac batteri,polish batteri metal,3.00,polisher battery metal,Lenmar Nickel-Metal Hydride 1200mAh/3.6-Volt Cordless Phone Replacement Battery,"power panasonic, american telecom, tele-phone, key phone, audiovox, uniden, universal, zeta cordless phone lenmar nickel-met hydrid cordless phone replac battery. this 3.6-volt/1200mah batteri provid reliable, long-last power. compat panason kx-a150, american telecom 2110, tele-phon tel-1000, key phone 920-wd, audiovox at-12a, uniden ex-3300, univers 2000, zeta 510.fits: panason kx-a150, american telecom 2110, tele-phon tel-1000, key phone 920-wd, audiovox at-12a, uniden ex-3300, univers 2000, zeta 510replaces: dantona 3aa, empir cpb-400, general electr bt-17, panason p-p341pa, recoton t121, uniden ba-3003.6-volt / 1200mahnickel-met hydride1 year warranti","Power your Panasonic, American Telecom, Tele-Phone, Key Phone, Audiovox, Uniden, Un

In [70]:
train['look_kit'] = train['search_term'].str.contains('kit', regex=False)
train['is_kit'] = train['product_title'].str.contains('kit', regex=False)

test['look_kit'] = test['search_term'].str.contains('kit', regex=False)
test['is_kit'] = test['product_title'].str.contains('kit', regex=False)


tree_cols = ['match_product_title', 'match_descr', 'len_search_term_words', 'len_search_term_letters',
             'match_pos_title', 'match_pos_descr', 'prod_title_ind', 'descr_ind',
             'words_std_title', 'words_std_descr', 'len_title', 'len_descr', 'last_word_title',
             'last_word_descr', 'match_orig_title', 'match_orig_descr',
             'fuzzy_title', 'fuzzy_descr', 'fuzzy_orig_title', 'fuzzy_orig_descr',
             'part_fuzzy_title', 'part_fuzzy_descr', 'part_fuzzy_orig_title', 'part_fuzzy_orig_descr',
             'first_word_title', 'first_word_descr',
             'match_numbers_title',
             'fuzzy_brand',
             'match_bullets', 'look_kit', 'is_kit']

use_cols = tree_cols + ['search_term', 'product_title', 'descr', 'brand', 'bullets']

clf = fitter()
X_train = clf.transform(train[use_cols])
X_test = clf.transform(test[use_cols])

gbm = lgb.LGBMRegressor(learning_rate=0.03, colsample_bytree=0.5, num_leaves=80, n_estimators=1000)
gbm.fit(X_train, y)

tc_len = len(tree_cols)
print(dict(zip(tree_cols, gbm.feature_importances_[:tc_len]) ))
print(gbm.feature_importances_[tc_len:])

write_submission(gbm, X_test)

{'match_product_title': 241, 'match_descr': 160, 'len_search_term_words': 225, 'len_search_term_letters': 589, 'match_pos_title': 143, 'match_pos_descr': 135, 'prod_title_ind': 594, 'descr_ind': 468, 'words_std_title': 227, 'words_std_descr': 254, 'len_title': 370, 'len_descr': 325, 'last_word_title': 94, 'last_word_descr': 61, 'match_orig_title': 236, 'match_orig_descr': 168, 'fuzzy_title': 585, 'fuzzy_descr': 352, 'fuzzy_orig_title': 514, 'fuzzy_orig_descr': 311, 'part_fuzzy_title': 140, 'part_fuzzy_descr': 137, 'part_fuzzy_orig_title': 179, 'part_fuzzy_orig_descr': 75, 'first_word_title': 99, 'first_word_descr': 78, 'match_numbers_title': 143, 'fuzzy_brand': 95, 'match_bullets': 195, 'look_kit': 1, 'is_kit': 3}
[393 388 397 305 316 354 294 273 288 246 318 268 291 302 334 294 303 251
 265 330 293 284 303 295 264 288 351 288 310 278 263 239 290 342 323 282
 307 293 314 372 278 263 272 285 279 258 299 301 332 353 269 308 236 270
 337 327 394 301 309 287 310 303 315 294 314 333 300 348 

0.46739

In [71]:
train['pred_diff'] = train['relevance'] - gbm.predict(X_train)
train.sort_values(['pred_diff'], ascending=False)[:50]

,product_uid,product_title,search_term,relevance,orig_search_term,orig_product_title,descr,orig_descr,match_product_title,match_descr,len_search_term_words,len_search_term_letters,complete_match_title,complete_match_descr,brand,match_brand,words_std_title,words_std_descr,last_word_title,last_word_descr,prod_title_ind,descr_ind,match_pos_title,match_pos_descr,match_orig_title,match_orig_descr,len_title,len_descr,fuzzy_title,fuzzy_descr,fuzzy_orig_title,fuzzy_orig_descr,part_fuzzy_title,part_fuzzy_descr,part_fuzzy_orig_title,part_fuzzy_orig_descr,pred_diff,first_word_title,first_word_descr,match_numbers_title,match_numbers_descr,fuzzy_brand,color,bullets,material,match_color,match_bullets,match_material,fuzzy_bullets,look_kit,is_kit
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
203914,192088,behr premium plus 600e 2 harbor mist zero voc interior paint,locit 2 plus 1,3.00,locite 2 plus 1,BEHR Premium Plus #600E-2 Harbor Mist Zero VOC Interior Paint,"behr premium plus zero voc, self-prim interior flat 100% acryl provid mildew resist finish. this easy-clean flat provid excel touch-up, ultim durabl except hide. the finish display non-reflective, matt appearance.california residents: see&nbsp;proposit 65 informationid famili rooms, live rooms, dine rooms, bedroom ceilings100% acryl finish easi cleaningtough durabl matt finishexcept hide, 1 cover 400 sq. ft. depend color select surfac porosityeasi soap water clean-upexcept latex formula provid mildew resist finishself prime finish seal surfaceactu paint color may vari on-screen printer representationsonlin price includ paint care fee follow states: ca, co, ct, me, mn, or, ri, vt","BEHR Premium Plus Zero VOC, Self-Priming Interior Flat is 100% acrylic and provides a mildew resistant finish. This easy-clean flat provides excellent touch-up, ultimate durability and exceptional hide. The finish displays a non-reflective, matte appearance.California residents: see&nbsp;Proposition 65 informationIdeal for family rooms, living rooms, dining rooms, bedrooms and ceilings100% acrylic finish for easy cleaningTough and durable matte finishExceptional hide, 1 can covers up to 400 sq. ft. depending on color selected and surface porosityEasy soap and water clean-upExceptional latex formula provides a mildew resistant finishSelf priming finish that seals the surfaceActual paint colors may vary from on-screen and printer representationsOnline Price includes Paint Care fee in the following states: CA, CO, CT, ME, MN, OR, RI, VT",0.500000,0.500000,4,14,0,0,behr premium plus,0.333333,1.00,576.00,0,1,0.181818,0.021053,0.500000,0.500000,0.250000,0.500000,11,95,60,60,57,57,100,100,100,100,1.136893,0,0,0,0,100,purpl lavend,ideal famili room live room dine room bedroom ceil 100 acryl finish easi clean tough durabl matt finish except hide 1 cover 400 sq. ft. depend color select surfac poros easi soap water clean except latex formula provid mildew resist finish self prime finish seal surfac actual paint color may vari screen printer represent,nan,0.0,0.018519,0.000000,100,False,False
194147,184559,lenmar nickel metal hydrid 1200mah 3.6 volt cordless phone replac batteri,polish batteri metal,3.00,polisher battery metal,Lenmar Nickel-Metal Hydride 1200mAh/3.6-Volt Cordless Phone Replacement Battery,"power panasonic, american telecom, tele-phone, key phone, audiovox, uniden, universal, zeta cordless phone lenmar nickel-met hydrid cordless phone replac battery. this 3.6-volt/1200mah batteri provid reliable, long-last power. compat panason kx-a150, american telecom 2110, tele-phon tel-1000, key phone 920-wd, audiovox at-12a, uniden ex-3300, univers 2000, zeta 510.fits: panason kx-a150, american telecom 2110, tele-phon tel-1000, key phone 920-wd, audiovox at-12a, uniden ex-3300, univers 2000, zeta 510replaces: dantona 3aa, empir cpb-400, general electr bt-17, panason p-p341pa, recoton t121, uniden ba-3003.6-volt / 1200mahnickel-met hydride1 year warranti","Power your Panasonic, American Telecom, Tele-Phone, K

In [72]:
train.sort_values(['pred_diff'], ascending=True)[:50]

,product_uid,product_title,search_term,relevance,orig_search_term,orig_product_title,descr,orig_descr,match_product_title,match_descr,len_search_term_words,len_search_term_letters,complete_match_title,complete_match_descr,brand,match_brand,words_std_title,words_std_descr,last_word_title,last_word_descr,prod_title_ind,descr_ind,match_pos_title,match_pos_descr,match_orig_title,match_orig_descr,len_title,len_descr,fuzzy_title,fuzzy_descr,fuzzy_orig_title,fuzzy_orig_descr,part_fuzzy_title,part_fuzzy_descr,part_fuzzy_orig_title,part_fuzzy_orig_descr,pred_diff,first_word_title,first_word_descr,match_numbers_title,match_numbers_descr,fuzzy_brand,color,bullets,material,match_color,match_bullets,match_material,fuzzy_bullets,look_kit,is_kit
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
69679,119478,romano 4ft. boxwood spiral topiari tree,topiari tree,1.00,topiary tree,Romano 4 ft. Boxwood Spiral Topiary Tree,"enhanc home romano boxwood spiral topiari tree. this wonder full bodi boxwood spiral topiari tree keep color rest lose color leaves. you alway look forward rich green tone romano boxwood spiral topiari tree home. if look high qualiti artifici tree make bold statement home, it.light mobil designov 700 leavesindoor/outdooriron frameuv resist rubber leaves7 in. potenh room hous","Enhance your home with a Romano Boxwood Spiral Topiary Tree. This wonderful full bodied Boxwood Spiral Topiary Tree keeps its colors when all the rest lose their color and leaves. You can always look forward to the rich green tones of the Romano Boxwood Spiral Topiary Tree in your home. If you are looking for a High quality artificial tree to make a bold statement in your home, this is it.Light and mobile designOver 700 leavesIndoor/outdoorIron frameUV resistant rubber leaves7 in. potEnhances any room in your house",1.000000,1.000000,2,12,1,1,NaN,0.0,0.250000,20.250000,1,1,0.666667,0.087719,1.000000,1.000000,1.000000,1.000000,6,57,100,100,100,100,100,100,100,100,-1.529709,1,1,0,0,0,nan,nan,nan,0.0,0.000000,0.0,33,False,False
136808,148897,earthquak 212cc tiller rear tine crt side shield,side shield,1.00,side shields,Earthquake 212cc Tiller Rear Tine CRT with Side Shields,"the earthquak 6015v rear tine rototil deliv ultim combin power size. rear-mount tine till stubborn dirt easili high-perform 212cc viper engine. we engin compact rear tine rototil power larger counterparts.easi - single-hand control sodearthquak 6015v rear tinecompact - full-siz power compact framedur - cast iron, bronz gear drive transmiss","The Earthquake 6015V rear tine rototiller delivers the ultimate combination of power and size. Rear-mounted tines till stubborn dirt easily with a high-performance 212cc Viper engine. We have engineered a compact rear tine rototiller with the same power as its larger counterparts.Easy - single-handed control through sodEarthquake 6015V rear tineCompact - full-sized power in a compact frameDurable - cast iron, bronze gear drive transmission",1.000000,0.000000,2,11,1,0,NaN,0.0,0.250000,0.000000,1,0,0.750000,1.000000,1.000000,0.000000,1.000000,0.000000,8,49,100,0,100,0,100,64,100,67,-1.399417,1,0,0,0,0,nan,nan,nan,0.0,0.000000,0.0,0,False,False
120624,140844,rachael ray 10 qt. cover stockpot,rachael ray,1.00,rachael ray,Rachael Ray 10 qt. Covered Stockpot,"whether boil pasta, make batch chili cook grains, large, 10 quart stockpot right size. when stovetop, two short side handl help take less space cabinet. the hard-anod construct prefer even heat distribut throughout pan sides. the nonstick cook surfac mean need worri food stick scrub dirti pan - cleanup breeze. in fact, rachael stockpot dishwash safe ad convenience.hard-anod cookwar except durable, plus heat quick evenly, reduc ""hot spots"" burn foodsa clear coat exterior make cookwar dishwash safe conveniencelong-last nonstick interior let food slide eas make cleanup quick easyfun rubber handl provid comfort grip oven safe 350 degre fahrenheittemp glass lid watch food cook without remov lid keep 

In [75]:
nltk.download('words')
from nltk.corpus import words

all_words = words.words()

[nltk_data] Downloading package words to /home/eugene/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


In [76]:
@np.vectorize
def is_typo(search_term):
    search_term = search_term.split()
    if len(search_term) == 0:
        return 0
    for word in search_term:
        if word in all_words:
            pass
        else:
            return 1
    return 0

train['typo_search_term'] = is_typo(train['orig_search_term'].str.lower())
test['typo_search_term'] = is_typo(test['orig_search_term'].str.lower())


In [77]:
tree_cols = ['match_product_title', 'match_descr', 'len_search_term_words', 'len_search_term_letters',
             'match_pos_title', 'match_pos_descr', 'prod_title_ind', 'descr_ind',
             'words_std_title', 'words_std_descr', 'len_title', 'len_descr', 'last_word_title',
             'last_word_descr', 'match_orig_title', 'match_orig_descr',
             'fuzzy_title', 'fuzzy_descr', 'fuzzy_orig_title', 'fuzzy_orig_descr',
             'part_fuzzy_title', 'part_fuzzy_descr', 'part_fuzzy_orig_title', 'part_fuzzy_orig_descr',
             'first_word_title', 'first_word_descr',
             'match_numbers_title',
             'fuzzy_brand',
             'match_bullets', 'look_kit', 'is_kit', 'typo_search_term']

use_cols = tree_cols + ['search_term', 'product_title', 'descr', 'brand', 'bullets']

clf = fitter()
X_train = clf.transform(train[use_cols])
X_test = clf.transform(test[use_cols])

gbm = lgb.LGBMRegressor(learning_rate=0.03, colsample_bytree=0.5, num_leaves=80, n_estimators=1000)
gbm.fit(X_train, y)

tc_len = len(tree_cols)
print(dict(zip(tree_cols, gbm.feature_importances_[:tc_len]) ))
print(gbm.feature_importances_[tc_len:])

write_submission(gbm, X_test)

{'match_product_title': 266, 'match_descr': 156, 'len_search_term_words': 250, 'len_search_term_letters': 627, 'match_pos_title': 127, 'match_pos_descr': 125, 'prod_title_ind': 553, 'descr_ind': 476, 'words_std_title': 236, 'words_std_descr': 217, 'len_title': 368, 'len_descr': 334, 'last_word_title': 94, 'last_word_descr': 76, 'match_orig_title': 224, 'match_orig_descr': 174, 'fuzzy_title': 573, 'fuzzy_descr': 408, 'fuzzy_orig_title': 496, 'fuzzy_orig_descr': 344, 'part_fuzzy_title': 163, 'part_fuzzy_descr': 123, 'part_fuzzy_orig_title': 193, 'part_fuzzy_orig_descr': 89, 'first_word_title': 76, 'first_word_descr': 57, 'match_numbers_title': 146, 'fuzzy_brand': 101, 'match_bullets': 193, 'look_kit': 2, 'is_kit': 6, 'typo_search_term': 80}
[429 419 386 319 351 332 300 278 322 248 296 275 268 353 340 284 300 278
 274 350 272 280 352 266 267 270 326 259 274 329 295 311 327 284 278 278
 330 287 325 288 316 252 267 313 325 263 331 285 335 267 305 322 256 292
 267 280 278 267 284 307 318 333

0.46748

In [78]:
train.to_csv('zip/train6.csv.gz', compression='gzip')
test.to_csv('zip/test6.csv.gz', compression='gzip')

In [79]:
train['look_part'] = train['search_term'].str.contains('(part|case|cover|tool|kit)')
train['is_part'] = train['product_title'].str.contains('(part|case|cover|tool|kit)')

test['look_part'] = test['search_term'].str.contains('(part|case|cover|tool)')
test['is_part'] = test['product_title'].str.contains('(part|case|cover|tool)')

tree_cols = ['match_product_title', 'match_descr', 'len_search_term_words', 'len_search_term_letters',
             'match_pos_title', 'match_pos_descr', 'prod_title_ind', 'descr_ind',
             'words_std_title', 'words_std_descr', 'len_title', 'len_descr', 'last_word_title',
             'last_word_descr', 'match_orig_title', 'match_orig_descr',
             'fuzzy_title', 'fuzzy_descr', 'fuzzy_orig_title', 'fuzzy_orig_descr',
             'part_fuzzy_title', 'part_fuzzy_descr', 'part_fuzzy_orig_title', 'part_fuzzy_orig_descr',
             'first_word_title', 'first_word_descr',
             'match_numbers_title',
             'fuzzy_brand',
             'match_bullets', 'typo_search_term',
             'look_part', 'is_part']

use_cols = tree_cols + ['search_term', 'product_title', 'descr', 'brand', 'bullets']

clf = fitter()
X_train = clf.transform(train[use_cols])
X_test = clf.transform(test[use_cols])

gbm = lgb.LGBMRegressor(learning_rate=0.03, colsample_bytree=0.5, num_leaves=80, n_estimators=1000)
gbm.fit(X_train, y)

tc_len = len(tree_cols)
print(dict(zip(tree_cols, gbm.feature_importances_[:tc_len]) ))
print(gbm.feature_importances_[tc_len:])

write_submission(gbm, X_test)

/home/eugene/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """Entry point for launching an IPython kernel.
/home/eugene/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  
/home/eugene/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  after removing the cwd from sys.path.
/home/eugene/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """


{'match_product_title': 250, 'match_descr': 162, 'len_search_term_words': 241, 'len_search_term_letters': 626, 'match_pos_title': 149, 'match_pos_descr': 120, 'prod_title_ind': 560, 'descr_ind': 453, 'words_std_title': 252, 'words_std_descr': 236, 'len_title': 385, 'len_descr': 373, 'last_word_title': 83, 'last_word_descr': 71, 'match_orig_title': 204, 'match_orig_descr': 143, 'fuzzy_title': 587, 'fuzzy_descr': 434, 'fuzzy_orig_title': 511, 'fuzzy_orig_descr': 352, 'part_fuzzy_title': 131, 'part_fuzzy_descr': 143, 'part_fuzzy_orig_title': 192, 'part_fuzzy_orig_descr': 85, 'first_word_title': 88, 'first_word_descr': 57, 'match_numbers_title': 158, 'fuzzy_brand': 88, 'match_bullets': 231, 'typo_search_term': 87, 'look_part': 6, 'is_part': 50}
[393 412 386 351 308 351 299 240 313 210 286 284 259 326 291 294 305 289
 270 269 255 311 283 275 272 239 352 236 322 284 281 324 309 297 323 279
 336 293 379 302 292 296 224 262 286 259 313 298 336 249 246 265 270 317
 319 317 311 297 290 328 316 3

In [80]:
train.to_csv('zip/train6.csv.gz', compression='gzip')
test.to_csv('zip/test6.csv.gz', compression='gzip')